# Code Generator

암호 최적화 문제의 정답 코드를 생성하기 위해 만든 노트북이다.

## Loop Unrolling
우선 loop unrolling으로 성능을 개선해 보자.

### `key_gen` 함수 최적화
원본코드를 우선 다음과 같이 가져오자.

In [1]:
original_code = '''key_in[0] = ROL8(key_in[0], 1) + ROL8(key_in[0], 5);
key_in[1] = ROL8(key_in[1], 3) + ROL8(key_in[1], 7);
tmp1 = key_in[0] + key_in[1];
tmp2 = key_in[0] ^ key_in[1];
key_in[0] = tmp1;
key_in[1] = tmp2;
rnd[{0} * 2 + 0] = key_in[0];
rnd[{0} * 2 + 1] = key_in[1];
*key_p = ROL16(*key_p, 1) + ROL16(*key_p, 9) + ROL16(con, (({0} + 1) % 16));
tmp1 = key_in[0] + key_in[1];
tmp2 = key_in[0] ^ key_in[1];
key_in[0] = tmp1;
key_in[1] = tmp2;
rnd[({0} + 1) * 2 + 0] = key_in[0];
rnd[({0} + 1) * 2 + 1] = key_in[1];'''

이제, 1부터 128까지의 코드를 채워야 한다. 코드가 참 길다.

In [2]:
unrolled = ''
for i in range(0, 128, 2):
    unrolled += original_code.format(i)
# print(unrolled) # not a good idea -- the code is *really* long
print('unrolled code length: %d' % len(unrolled))

unrolled code length: 32109


노트북에 프린트할 수는 없으니, 파일로 저장하자.

In [3]:
with open('unrolled_key_gen.txt', 'w') as f:
    f.write(unrolled)

실험 결과, 원래 코드의 3909ms에서 1923ms까지, 2배에 가까운 성능 향상을 얻었다.

### `enc` 함수 최적화
`enc` 함수에도 고정된 횟수로 반복하는 루프가 존재한다. 코드를 가져오자.

In [4]:
original_enc_code = '''*text_p = ROL16(*text_p, 4);
text_in[0] = text_in[0] + rnd[{0} * 2 + 0];
text_in[1] = text_in[1] ^ rnd[{0} * 2 + 1];
*text_p = ROL16(*text_p, 8);
text_in[0] = text_in[0] + rnd[({0} + 1) * 2 + 0];
text_in[1] = text_in[1] ^ rnd[({0} + 1) * 2 + 1];'''

방금 전처럼 128개를 채워주자.

In [5]:
unrolled_enc = ''
for i in range(0, 128, 2):
    unrolled_enc += original_enc_code.format(i)
print('unrolled code length: %d' % len(unrolled_enc))

unrolled code length: 15460


코드를 저장하자.

In [6]:
with open('unrolled_enc.txt', 'w') as f:
    f.write(unrolled_enc)

의외로, 전체 루프를 풀었을 때 1648ms가 나오고, 루프 내부에서 `if`만 제거했을 때 1641ms가 나왔다. 이 부분은 컴파일러가 수동 최적화보다 잘 하는것으로 추정된다. 아마 `-O3` 컴파일 옵션의 영향일 것이다.

따로 첨부된 코드의 `enc` 함수에서는 이러한 이유로 여기서 unrolling한 코드 대신 루프 내부의 `if`문만 제거했다.